### Import

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

import lightgbm as lgb
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import mutual_info_classif

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

train = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./data/test.csv').drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

# 수치형 컬럼들을 0으로 채움
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# categorical_feature에 인덱스를 전달
categorical_feature_indices = [X_train_encoded.columns.get_loc(col) for col in categorical_columns]

# 스케일링 적용
scaler = StandardScaler()
X_train_encoded[numeric_columns] = scaler.fit_transform(X_train_encoded[numeric_columns])
X_test_encoded[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

# # 데이터 불균형 확인 및 SMOTE 적용
# smote = SMOTE(random_state=42)
# X_train_encoded, y = smote.fit_resample(X_train_encoded, y)

# 데이터 불균형 확인 및 SMOTE + Tomek 적용
smt = SMOTETomek(random_state=42)
X_train_encoded, y = smt.fit_resample(X_train_encoded, y)

# Feature Selection (SelectKBest)
selector = SelectKBest(score_func=mutual_info_classif, k=50)  # 가장 중요한 30개의 변수를 선택
X_train_encoded = selector.fit_transform(X_train_encoded, y)
X_test_encoded = selector.transform(X_test_encoded)

# -------------------------- 3. 하이퍼파라미터 튜닝 --------------------------
# LightGBM 하이퍼파라미터 그리드 (scikit-learn API 사용)
lgb_param_grid = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 70],
    'max_depth': [10, 15, -1]
}

lgb_est = LGBMClassifier(random_state=42, objective='binary')
grid_lgb = GridSearchCV(lgb_est, lgb_param_grid, scoring='roc_auc', cv=3, n_jobs=-1, verbose=1)
grid_lgb.fit(X_train_encoded, y)
best_lgb = grid_lgb.best_estimator_
print("LightGBM 최적 파라미터:", grid_lgb.best_params_)

# XGBoost 하이퍼파라미터 그리드
xgb_param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5]
}

xgb_est = XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)
grid_xgb = GridSearchCV(xgb_est, xgb_param_grid, scoring='roc_auc', cv=3, n_jobs=-1, verbose=1)
grid_xgb.fit(X_train_encoded, y)
best_xgb = grid_xgb.best_estimator_
print("XGBoost 최적 파라미터:", grid_xgb.best_params_)
print("________________________________________________________________________________ \n")

# -------------------------- 4. Stacking Ensemble 구성 --------------------------

# StackingClassifier: base estimator로 최적의 LightGBM과 XGBoost, 메타 모델로 LogisticRegression 사용
stacking_clf = StackingClassifier(
    estimators=[('lgb', best_lgb), ('xgb', best_xgb)],
    final_estimator=LogisticRegression(random_state=42),
    cv=3,
    n_jobs=-1,
    passthrough=True
)

# -------------------------- 5. 교차 검증을 통한 성능 비교 --------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_lgb = []
cv_scores_xgb = []
cv_scores_stack = []

# 교차 검증을 위해 numpy 배열로 변환 (SelectKBest 결과는 numpy array)
X_fs = X_train_encoded  # 이미 numpy array
y_array = y.values  # numpy array로 변환

for train_idx, val_idx in cv.split(X_fs, y_array):
    X_cv_train, X_cv_val = X_fs[train_idx], X_fs[val_idx]
    y_cv_train, y_cv_val = y_array[train_idx], y_array[val_idx]
    
    # 개별 모델 예측 (최적 모델로 재학습)
    best_lgb.fit(X_cv_train, y_cv_train)
    lgb_proba = best_lgb.predict_proba(X_cv_val)[:, 1]
    auc_lgb = roc_auc_score(y_cv_val, lgb_proba)
    cv_scores_lgb.append(auc_lgb)
    
    best_xgb.fit(X_cv_train, y_cv_train)
    xgb_proba = best_xgb.predict_proba(X_cv_val)[:, 1]
    auc_xgb = roc_auc_score(y_cv_val, xgb_proba)
    cv_scores_xgb.append(auc_xgb)
    
    # Stacking 앙상블 예측
    stacking_clf.fit(X_cv_train, y_cv_train)
    stack_proba = stacking_clf.predict_proba(X_cv_val)[:, 1]
    auc_stack = roc_auc_score(y_cv_val, stack_proba)
    cv_scores_stack.append(auc_stack)

print("------------- 교차 검증 평균 ROC-AUC -------------")
print(f"LightGBM: {np.mean(cv_scores_lgb):.4f}")
print(f"XGBoost: {np.mean(cv_scores_xgb):.4f}")
print(f"Stacking Ensemble: {np.mean(cv_scores_stack):.4f}")
print("---------------------------------------------------\n")

# -------------------------- 6. 최종 모델 학습 및 제출 파일 생성 --------------------------

# 전체 학습 데이터(Feature Selection 결과)로 Stacking 모델 학습
stacking_clf.fit(X_train_encoded, y)

# 테스트 데이터에 대한 예측 (Stacking 앙상블)
final_pred_proba = stacking_clf.predict_proba(X_test_encoded)[:, 1]

# 제출 파일 생성
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = final_pred_proba
sample_submission.to_csv('./submit/stacking_ensemble_submit.csv', index=False)

print("최종 제출 파일 'stacking_ensemble_submit.csv' 생성 완료!")

Fitting 3 folds for each of 54 candidates, totalling 162 fits


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn

[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.179847 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10556
[LightGBM] [Info] Number of data points in the train set: 246608, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.100135 seconds.
You can set `fo

/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `skl

[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.270852 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11103
[LightGBM] [Info] Number of data points in the train set: 246608, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.298197 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10546
[LightGBM] [Info] Number of data points in the train set: 246608, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> inits

/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `skl

[LightGBM] [Info] Number of positive: 123304, number of negative: 123304
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.230608 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10556
[LightGBM] [Info] Number of data points in the train set: 246608, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
